In [177]:
import pandas as pd
import rdflib

g = rdflib.Graph()
result = g.parse("mhdb.ttl", format='ttl')

In [3]:
result

<Graph identifier=N4bf769019943487f92611e1c74086489 (<class 'rdflib.graph.Graph'>)>

In [172]:
g

<Graph identifier=N85ec1cd6d02a47788ac1522a2b7d9166 (<class 'rdflib.graph.Graph'>)>

In [6]:
for s, p, o in g:
    print(" ".join([s, p, o, "\n"]))

http://www.purl.org/mentalhealth#20_Over_time_I_have_found_that_I_need_to_eat_more_and_more_to_get_the_feeling_I_want_such_as_reduced_negative_emotions_or_increased_pleasure http://www.w3.org/2000/01/rdf-schema#label 20. Over time, I have found that I need to eat more and more to get the feeling I want, such as reduced negative emotions or increased pleasure. 

http://www.purl.org/mentalhealth#Disobeys http://dublincore.org/documents/2012/06/14/dcmi-terms/isPartOf http://www.purl.org/mentalhealth#BASC-3_Behaviour_Assessment_System_for_Children 

http://www.purl.org/mentalhealth#I_thought_my_family_would_be_better_off_without_me http://dublincore.org/documents/2012/06/14/dcmi-terms/isPartOf http://www.purl.org/mentalhealth#Mood_and_Feelings_Questionnaire_MFQ_Child_Report 

http://www.purl.org/mentalhealth#Other_Hallucinogen_Use_Disorder http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class 

http://www.purl.org/mentalhealth#Sexual_activities http://www.w3.o

http://www.purl.org/mentalhealth#My_child_is_underweight_for_hisher_age_and_height http://www.purl.org/mentalhealth#hasAnswerType http://www.purl.org/mentalhealth#yesno 

http://www.purl.org/mentalhealth#When_Im_upset_I_become_embarrassed_for_feeling_that_way http://dublincore.org/documents/2012/06/14/dcmi-terms/isPartOf http://www.purl.org/mentalhealth#DERS-Difficulties_in_Emotion_Regulation_Scale 

http://www.purl.org/mentalhealth#26_Your_mom_goes_to_a_meeting_at_school_like_a_PTA_meeting_or_a_parentteacher_conference http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.purl.org/mentalhealth#Question 

http://www.purl.org/mentalhealth#Which_one_of_the_following_describes_you_best_for_the_last_7_days_Read_all_five_statements_before_deciding_on_the_one_answer_that_describes_you http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.purl.org/mentalhealth#Question 

http://www.purl.org/mentalhealth#Sasses_teacher http://dublincore.org/documents/2012/06/14/dcmi-terms/isPartOf ht


http://www.purl.org/mentalhealth#I_feel_lonely http://dublincore.org/documents/2012/06/14/dcmi-terms/isPartOf http://www.purl.org/mentalhealth#CBCL_Self_11-18 

http://www.purl.org/mentalhealth#I_wished_that_I_had_the_right_to_kill_myself http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.purl.org/mentalhealth#Question 

http://www.purl.org/mentalhealth#with_your_behavior_at_home http://www.w3.org/2000/01/rdf-schema#label with your behavior at home 

http://www.purl.org/mentalhealth#Makes_friends_easily http://dublincore.org/documents/2012/06/14/dcmi-terms/isPartOf http://www.purl.org/mentalhealth#BASC-3_Behaviour_Assessment_System_for_Children 

http://www.purl.org/mentalhealth#Tobacco_Withdrawal http://www.purl.org/mentalhealth#hasICD9Code http://purl.bioontology.org/ontology/ICD9CM/292 

http://www.purl.org/mentalhealth#During_the_past_3_months_how_often_has_your_use_of_first_drug_second_drug_etc_led_to_health_social_legal_or_financial_problems_Sedatives_or_sleeping_pills_V


http://www.purl.org/mentalhealth#Do_you_own_or_rent_your_residence http://dublincore.org/documents/2012/06/14/dcmi-terms/isPartOf http://www.purl.org/mentalhealth#Financial_Support_Questionnaire_FSQ 

http://www.purl.org/mentalhealth#26m_Do_you_have_problems_with_Stawberries http://www.w3.org/2000/01/rdf-schema#label 26m. Do you have problems with Stawberries 

http://www.purl.org/mentalhealth#I_cant_handle_feeling_distressed_or_upset http://dublincore.org/documents/2012/06/14/dcmi-terms/isPartOf http://www.purl.org/mentalhealth#Distress_Tolerance_Scale_DTS 

http://www.purl.org/mentalhealth#Argues_with_teachers_about_rules_or_instructions http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.purl.org/mentalhealth#Question 

http://www.purl.org/mentalhealth#Excessive_concern_with_body_part_or_aspect_of_appearance_eg_dysmorphophobia http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.purl.org/mentalhealth#Question 

http://www.purl.org/mentalhealth#Blurts_out_answers_to_que

http://www.purl.org/mentalhealth#39_My_child_has_trouble_making_up_hisher_mind_about_simple_things http://www.w3.org/2000/01/rdf-schema#label 39. My child has trouble making up his/her mind about simple things 

http://www.purl.org/mentalhealth#4d_On_how_many_occasions_OVER_THE_LAST_30_DAYS_have_you_used_amphetamines_speed http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.purl.org/mentalhealth#Question 

http://www.purl.org/mentalhealth#Fails_to_pay_hisher_debts_or_meet_other_financial_responsibilities http://www.purl.org/mentalhealth#hasAnswerType http://www.purl.org/mentalhealth#ordinal-probabilityfrequencyintensity 

http://www.purl.org/mentalhealth#34_My_hands_shake http://www.w3.org/2000/01/rdf-schema#label 34. My hands shake 

http://www.purl.org/mentalhealth#43_Gets_out_of_control_more_than_playmates http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.purl.org/mentalhealth#Question 

http://www.purl.org/mentalhealth#Other_Cannabis-Induced_Disorders http://www.w3.

http://www.purl.org/mentalhealth#Tries_to_be_fair_to_others http://www.w3.org/2000/01/rdf-schema#label Tries to be fair to others 

http://www.purl.org/mentalhealth#18_I_have_trouble_getting_started_on_a_task http://www.purl.org/mentalhealth#hasAnswerType http://www.purl.org/mentalhealth#ordinal-frequency 

http://www.purl.org/mentalhealth#Thoughts_that_occur_over_and_over_again_that_are_unwanted http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.purl.org/mentalhealth#Obsessions 

http://www.purl.org/mentalhealth#Use_causes_clinically_significant_impairment http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.purl.org/mentalhealth#Question 

http://www.purl.org/mentalhealth#Does_shehe_ever_use_socially_inappropriate_questions_or_statements_For_example_does_shehe_ever_regularly_ask_personal_questions_or_make_personal_comments_at_awkward_times http://dublincore.org/documents/2012/06/14/dcmi-terms/isPartOf http://www.purl.org/mentalhealth#Social_Communication_Questionnaire_S

http://www.purl.org/mentalhealth#20_How_often_does_your_child_feel_depressed_moody_or_nervous_when_off-line_which_seems_to_goes_away_once_back_online http://dublincore.org/documents/2012/06/14/dcmi-terms/isPartOf http://www.purl.org/mentalhealth#Parent-Child_Internet_Addiction_Test_PCIAT 

http://www.purl.org/mentalhealth#My_child_feels_like_heshe_has_to_do_certain_things_in_a_very_specific_way_handwashing_checking_doing_things_multiple_times http://www.w3.org/2000/01/rdf-schema#label My child feels like he/she has to do certain things in a very specific way (handwashing, checking, doing things multiple times) 

http://www.purl.org/mentalhealth#8o_Have_you_ever_neglected_your_family_or_missed_work_because_of_your_use_of_cocaine http://dublincore.org/documents/2012/06/14/dcmi-terms/isPartOf http://www.purl.org/mentalhealth#The_European_School_Survey_Project_on_Alcohol_and_Other_Drugs_ESPAD 

http://www.purl.org/mentalhealth#My_child_smokes_or_chews_tobacco http://www.purl.org/mentalheal

http://www.purl.org/mentalhealth#Is_unable_to_slow_down http://www.purl.org/mentalhealth#hasAnswerType http://www.purl.org/mentalhealth#ordinal-frequency 

http://www.purl.org/mentalhealth#4e_On_how_many_occasions_OVER_THE_LAST_WEEK_have_you_used_amphetamines_speed http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.purl.org/mentalhealth#Question 

http://www.purl.org/mentalhealth#37_My_hands_feel_sweaty_or_cold http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.purl.org/mentalhealth#Question 

http://www.purl.org/mentalhealth#Bragging_boasting http://www.w3.org/2000/01/rdf-schema#label Bragging, boasting 

http://www.purl.org/mentalhealth#Threatens_to_hurt_others http://www.purl.org/mentalhealth#hasAnswerType http://www.purl.org/mentalhealth#ordinal-frequency 

http://www.purl.org/mentalhealth#can_be_with_other_children_but_only_on_hisher_terms http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.purl.org/mentalhealth#Question 

http://www.purl.org/mentalhealth#22


http://www.purl.org/mentalhealth#30_My_child_get_scared_riding_in_the_car_or_on_the_bus http://dublincore.org/documents/2012/06/14/dcmi-terms/isPartOf http://www.purl.org/mentalhealth#MASC2_Parent 

http://www.purl.org/mentalhealth#Sibling_Relational_Problem http://www.purl.org/mentalhealth#hasICD10Code http://purl.bioontology.org/ontology/ICD10CM/Z62.891 

http://www.purl.org/mentalhealth#14q_Have_you_ever_lost_a_job_because_of_GHB_abuse http://dublincore.org/documents/2012/06/14/dcmi-terms/isPartOf http://www.purl.org/mentalhealth#The_European_School_Survey_Project_on_Alcohol_and_Other_Drugs_ESPAD 

http://www.purl.org/mentalhealth#Schizophrenia http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class 

http://www.purl.org/mentalhealth#Poor_work_performance http://www.w3.org/2000/01/rdf-schema#label Poor work performance 

http://www.purl.org/mentalhealth#Perseverative_or_repetitive_action__play__behavior_note_if_2_or_more_components_then_it_is_a_routine_a

http://www.purl.org/mentalhealth#color_time_tables_historical_events http://www.w3.org/2000/01/rdf-schema#label color; time tables; historical events 

http://www.purl.org/mentalhealth#Childhood-onset_type http://www.w3.org/2000/01/rdf-schema#label Childhood-onset type 

http://www.purl.org/mentalhealth#Threatens_to_hurt_people http://www.purl.org/mentalhealth#hasAnswerType http://www.purl.org/mentalhealth#ordinal-probabilityfrequencyintensity 

http://www.purl.org/mentalhealth#Intensity_of_ideation_The_following_feature_should_be_rated_with_respect_to_the_most_severe_type_of_ideation_ie_1-5_from_above_with_1_being_the_least_severe http://dublincore.org/documents/2012/06/14/dcmi-terms/isPartOf http://www.purl.org/mentalhealth#Columbia_Suicide_Severity_Rating_Scale 

http://www.purl.org/mentalhealth#ordinal-intensity http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.purl.org/mentalhealth#AnswerType 

http://www.purl.org/mentalhealth#Active_Suicidal_Ideation_with_Specific_Plan_a

In [168]:
qres ="""PREFIX : <http://www.purl.org/mentalhealth#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?option ?subclass 
WHERE {
            ?subclass rdfs:subClassOf* ?intermediate .
            ?intermediate rdfs:subClassOf* :MentalDisorder .
            ?subclass rdfs:label ?label .
            BIND ( IF ( !bound(?label), ?subclass, IF ( bound(?label), ?label, '' ) ) AS ?option )
}
GROUP BY ?option ?subclass
HAVING(count(?intermediate) = 2)
ORDER BY ?option
        """
column_names = ['label', 'iri']
sparql_result = g.query(qres)
options = []
for row in sparql_result:
    options.append([row[0], row[1]])
df = pd.DataFrame(options, columns=column_names)
for key, value in df.iterrows():
    print(value['iri'])

http://www.purl.org/mentalhealth#Anxiety_Disorder
http://www.purl.org/mentalhealth#Bipolar_or_Related_Disorder
http://www.purl.org/mentalhealth#Depressive_Disorder
http://www.purl.org/mentalhealth#Disruptive_Impulse-Control_or_Conduct_Disorder
http://www.purl.org/mentalhealth#Dissociative_Disorder
http://www.purl.org/mentalhealth#Elimination_Disorder
http://www.purl.org/mentalhealth#Feeding_or_Eating_Disorder
http://www.purl.org/mentalhealth#Gender_Dysphoria
http://www.purl.org/mentalhealth#Medication-Induced_Movement_Disorder_or_Other_Adverse_Effects_of_Medication
http://www.purl.org/mentalhealth#Neurocognitive_Disorder
http://www.purl.org/mentalhealth#Obsessive-Compulsive_or_Related_Disorder
http://www.purl.org/mentalhealth#Other_Conditions_That_May_Be_a_Focus_of_Clinical_Attention
http://www.purl.org/mentalhealth#Other_Mental_Disorder
http://www.purl.org/mentalhealth#Paraphilic_Disorder
http://www.purl.org/mentalhealth#Personality_Disorder
http://www.purl.org/mentalhealth#Schizophre

In [191]:
qres2 = """PREFIX : <http://www.purl.org/mentalhealth#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?subject ?relationship ?object WHERE {
       { :Specific_Phobia ?relationship ?object }
       UNION
       { ?subject ?relationship :Specific_Phobia }
       BIND ( IF ( bound(?subject), ?subject, :Specific_Phobia ) AS ?subject )
       BIND ( IF ( bound(?object), ?object, :Specific_Phobia ) AS ?object )
}
        """
sparql_result = g.query(qres2)
options = []
for row in sparql_result:
    options.append([[row[0]], [row[1]], [row[2]]])
df = pd.DataFrame(options, columns=['subject', 'relationship', 'object'])
print(df)

                                             subject  \
0  [http://www.purl.org/mentalhealth#Specific_Pho...   
1  [http://www.purl.org/mentalhealth#Specific_Pho...   
2  [http://www.purl.org/mentalhealth#Specific_Pho...   

                                        relationship  \
0     [http://www.purl.org/mentalhealth#hasICD9Code]   
1  [http://www.w3.org/1999/02/22-rdf-syntax-ns#type]   
2       [http://www.w3.org/2000/01/rdf-schema#label]   

                                              object  
0  [http://purl.bioontology.org/ontology/ICD9CM/3...  
1              [http://www.w3.org/2002/07/owl#Class]  
2                                  [Specific Phobia]  


In [60]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.addDefaultGraph("mhdb.ttl")
sparql.setQuery(qres)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["label"]["value"])

KeyError: 'label'

In [42]:
from rdflib import Namespace

n = Namespace("mhdb.ttl")

In [46]:
from rdflib.namespace import *
OWL.Thing

rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Thing')